<h1 align="center"> Sensordatenfusion Tutorium 03 </h1>

<h3 align="center"> Erstellen der Prädiktion </h3>

Die Prädiktion hat den Zweck, dass wir eine Zustandsschätzung basierend auf $x_{k-1|k-1}$ erhalten.
Für $x_{k|k-1}$ benötigen wir lediglich noch unsere Evolutions-/Dynamikmatrix $F_{k|k-1}$, die im Zuge des zweiten Tutoriums erstellt wurde .
Für die dazugehörige Kovarianz $P_{k|k-1}$ brauchen wir zusätzlich zu $F$ noch das Prozessrauschen $Q_{k|k-1}$ (in den Folien $D_{k|k-1}$).
Ziel dieses Aufgabenteils ist es also, einen eigenen $Predictor$ zu erstellen, um $x_{k|k-1}$ und $P_{k|k-1}$ zu erhalten. 


Notwendige Imports 


In [ ]:
import scipy as sp
import numpy as np
from stonesoup.base import Property
from stonesoup.base import Predictor
from stonesoup.functions import gauss2sigma, unscented_transform
from stonesoup.types.prediction import GaussianStatePrediction
from stonesoup.types.state import State

In [ ]:
class SdfKalmanPredictor(Predictor)
    @lru_cache()
    def predict(self, prior, control_input=None, timestamp=None, **kwargs):
        
        
        # Compute time_interval
        try:
            time_interval = timestamp - prior.timestamp
        except TypeError:
            # TypeError: (timestamp or prior.timestamp) is None
            time_interval = None
            
            
        # Transition model parameters
        transition_matrix = self.transition_model.matrix(
            timestamp=timestamp,
            time_interval=time_interval,
            **kwargs)
        transition_noise_covar = self.transition_model.covar(
            timestamp=timestamp,
            time_interval=time_interval,
            **kwargs)
        
        # Control model parameters
        if self.control_model is None:
            control_matrix = np.zeros(prior.covar.shape)
            contol_noise_covar = np.zeros(prior.covar.shape)
            control_input = State(np.zeros(prior.state_vector.shape))
        else:
            # Extract control matrix
            control_matrix = self.control_model.matrix(
                timestamp=timestamp,
                time_interval=time_interval,
                **kwargs)
            # Extract control noise covariance
            try:
                # covar() is implemented for control_model
                control_noise_covar = self.control_model.covar(
                    timestamp=timestamp,
                    time_interval=time_interval,
                    **kwargs)
            except AttributeError:
                # covar() is NOT implemented for control_model
                contol_noise_covar = np.zeros(self.control_model.ndim_ctrl)
            if control_input is None:
                control_input = np.zeros((self.control_model.ndim_ctrl, 1))
                
        # Perform prediction
        prediction_mean, prediction_covar = self.predict_lowlevel(
            prior.mean, prior.covar, transition_matrix,
            transition_noise_covar, control_input.state_vector,
            control_matrix, contol_noise_covar)
        
        return GaussianStatePrediction(prediction_mean,
                                       prediction_covar,
                                       timestamp)
    
